In [1]:
import tensorflow as tf
import numpy as np
import os, shutil
from datetime import datetime as dt

In [10]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 3 
y = y.reshape(-1, 1)

In [20]:
tf.reset_default_graph()
layers      = [5, 3, 1]
activations = [tf.tanh, tf.tanh,  None]

inp = tf.placeholder(dtype=tf.float32, shape=(None, 2 ), name='inp')
out = tf.placeholder(dtype=tf.float32, shape=(None, 1 ), name='out')

isTraining = tf.placeholder(dtype=tf.bool, shape=(), name='isTraining')

N = inp * 1 # I am lazy
for i, (l, a) in enumerate(zip(layers, activations)):
    N = tf.layers.dense(N, l, None)
    N = tf.layers.batch_normalization( N,  training = isTraining) # comment this line
    if a is not None:
        N = a(N)

err = tf.reduce_mean((N - out)**2)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    opt = tf.train.AdamOptimizer(0.05).minimize(err)

tVars = tf.trainable_variables()
graph = tf.get_default_graph()
for v in graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES): 
    if all([
            ('batch_normalization' in v.name),
            ('optimizer' not in v.name), 
            v not in tVars ]):
        tVars.append(v)

init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list= tVars)

In [21]:
with tf.Session() as sess:
    sess.run(init)
    print('During Training')
    for i in range(3000):
        _, errVal = sess.run([opt, err], feed_dict={inp:X, out:y, isTraining:True})
        if i %500 == 0:
            print(errVal)

    shutil.rmtree('models1', ignore_errors=True)
    os.makedirs('models1')
    path = saver.save( sess, 'models1/model.ckpt' )

print('During testing')
with tf.Session() as sess:
    saver.restore(sess, path)
    errVal = sess.run(err, feed_dict={inp:X, out:y, isTraining:False})
    print( errVal )

During Training
31.7372
1.92066e-05
3.87879e-06
2.55274e-06
1.25418e-06
1.43078e-06
During testing
INFO:tensorflow:Restoring parameters from models1/model.ckpt
0.041519
